# Advanced Querying Mongo

**⚠️ IMPORTANT: Limit your print to avoid infinite scrolling. Otherwise your
code will be lost between print lines. If working with lists do:**

```python
list(collection.find(query))[:5] #or a reasonably low number
```

Importing libraries and setting up connection

In [50]:
from pymongo import MongoClient
import pandas as pd
import time
client = MongoClient("localhost:27017")
db = client["Ironhack"]
col = db.get_collection("companies")
col

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Ironhack'), 'companies')

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [8]:
list(col.find({'name': 'Babelgum'},{'_id':0, 'name':1}))

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [55]:
list(col.find({'number_of_employees':{"$gt":5000}},{'_id':0, 'name': 1}).sort('number_of_employees',-1).limit(20))

[{'name': 'Siemens'},
 {'name': 'IBM'},
 {'name': 'Toyota'},
 {'name': 'PayPal'},
 {'name': 'Nippon Telegraph and Telephone Corporation'},
 {'name': 'Samsung Electronics'},
 {'name': 'Accenture'},
 {'name': 'Tata Consultancy Services'},
 {'name': 'Flextronics International'},
 {'name': 'Safeway'},
 {'name': 'Sony'},
 {'name': 'LG'},
 {'name': 'Ford'},
 {'name': 'Boeing'},
 {'name': 'Digital Equipment Corporation'},
 {'name': 'Nokia'},
 {'name': 'MItsubishi Electric'},
 {'name': 'MItsubishi Electric'},
 {'name': 'Bertelsmann'},
 {'name': 'Comcast'}]

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [18]:
cond_2000= {'founded_year': {'$gte':2000}}
cond_2005= {'founded_year': {'$lte':2005}}
retrieve = {'name':1, '_id':0, 'founded_year':1}

companies= list(col.find({
    '$and':
    [cond_2000,cond_2005]},retrieve).limit(10))
companies

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Omnidrive', 'founded_year': 2005},
 {'name': 'StumbleUpon', 'founded_year': 2002},
 {'name': 'Gizmoz', 'founded_year': 2003},
 {'name': 'Helio', 'founded_year': 2005},
 {'name': 'Plaxo', 'founded_year': 2002},
 {'name': 'Technorati', 'founded_year': 2002}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

cond_1: Valuation Amount > 100.000.000 &
cond_2: founded lt>2010
retrv: name, ipo

In [57]:
cond_1 = {'ipo.valuation_amount':{'$gt':100000000}}
cond_2 = {'founded_year':{'$lt': 2010}}
retrieve ={'name':1, '_id':0, 'ipo':1}

list(col.find({
    '$and':
    [cond_1,cond_2]},retrieve).limit(5))


[{'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'name': 'Yelp',
  'ipo': {'valuation_amount': 1300000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 3,
   'pub_day': 2,
   'stock_symbol': 'NYSE:YELP'}},
 {'name': 'LinkedIn',
  'ipo': {'valuation_amount': 9310000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2011,
   'pub_month': 7,
   'pub_day': 20,
   'stock_symbol': 'NYSE:LNKD'}},
 {'name': 'Amazon',
  'ipo': {'valuation_amount': 100000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 1997,
   'pub_month': 5,
   'pub_day': None,
   'stock_symbol': 'NASDAQ:AMZN'}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

cond_1: employees lt 1000 & 
cond_2: founded lt 2005
sort by no. employees and limit 10

In [28]:
cond_1 = {'number_of_employees':{'$lt':1000}}
cond_2 = {'founded_year':{'$lt':2005}}
retrieve = {'name':1, '_id':0}


list(col.find({
    '$and':
    [cond_1,cond_2]},retrieve).sort("number_of_employees", -1).limit(10))


[{'name': 'Infinera Corporation'},
 {'name': 'NorthPoint Communications Group'},
 {'name': '888 Holdings'},
 {'name': 'Forrester Research'},
 {'name': 'Webmetrics'},
 {'name': 'SonicWALL'},
 {'name': 'Cornerstone OnDemand'},
 {'name': 'Buongiorno'},
 {'name': 'Cvent'},
 {'name': 'ZoomInfo'}]

### 6.-All-the-companies-that-don't-include-the-partners-field.

In [70]:
list(col.find({"partners": {"$exists": False}}).limit(10))

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [71]:
cond_1= {'category_code':{"$type": 'null'}}
retrieve = {'name':1, '_id':0}
list(col.find(cond_1,retrieve).limit(10))

[{'name': 'Collective'},
 {'name': 'Snimmer'},
 {'name': 'KoolIM'},
 {'name': 'Level9 Media'},
 {'name': 'VidKing'},
 {'name': 'Drigg'},
 {'name': 'SpaceTime'},
 {'name': 'Touch Clarity'},
 {'name': 'MMDAYS'},
 {'name': 'Inside Group'}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

employees lt 1000 &
employees gt 100 &
print name and n.o.e


In [37]:
cond_1 = {'number_of_employees':{'$lt':1000}}
cond_2 = {'number_of_employees':{'$gt':100}}
retrieve = {'name':1, '_id':0, 'number_of_employees':1}
list(col.find({
    '$and':
    [cond_1,cond_2]},retrieve).limit(10))

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110},
 {'name': 'Livestream', 'number_of_employees': 120},
 {'name': 'Ustream', 'number_of_employees': 250},
 {'name': 'iContact', 'number_of_employees': 300},
 {'name': 'Yelp', 'number_of_employees': 800},
 {'name': 'Dailymotion', 'number_of_employees': 120}]

### 9. Order all the companies by their IPO price in a descending order.

In [80]:

list(col.find({'ipo.valuation_amount': {"$exists": True}},{'name':1, '_id':0}).limit(5).sort('ipo.valuation_amount',-1))

[{'name': 'GREE'},
 {'name': 'Facebook'},
 {'name': 'Amazon'},
 {'name': 'Twitter'},
 {'name': 'Groupon'}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [83]:
list(col.find({'name': {"$exists": True}},{'name':1, '_id':0}).sort('number of employees',-1).limit(10))

[{'name': 'Omnidrive'},
 {'name': 'Fox Interactive Media'},
 {'name': 'Zoho'},
 {'name': 'Postini'},
 {'name': 'Flektor'},
 {'name': 'Facebook'},
 {'name': 'Wetpaint'},
 {'name': 'AdventNet'},
 {'name': 'Digg'},
 {'name': 'Geni'}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

founded_month gte 6  &
founded_month lte 12

In [44]:
cond_1: {'founded_month': {'$gte':6}}
cond_2: {'founded_month': {'$lte':12}}
list(col.find({
    '$and':
    [cond_1,cond_2]},{'name':1,'_id':0}).limit(1000))

[{'name': 'AdventNet'},
 {'name': 'AddThis'},
 {'name': 'OpenX'},
 {'name': 'LifeLock'},
 {'name': 'Jajah'},
 {'name': 'Livestream'},
 {'name': 'Ustream'},
 {'name': 'iContact'},
 {'name': 'Yelp'},
 {'name': 'Dailymotion'},
 {'name': 'RockYou'},
 {'name': 'Meebo'},
 {'name': 'Eventbrite'},
 {'name': 'Box'},
 {'name': 'Conduit'},
 {'name': 'oDesk'},
 {'name': 'PhotoBox'},
 {'name': 'Spreadshirt'},
 {'name': 'Bazaarvoice'},
 {'name': 'spigit'},
 {'name': 'PowerReviews'},
 {'name': 'Tagged'},
 {'name': 'PayScale'},
 {'name': 'Truphone'},
 {'name': 'Brightcove'},
 {'name': 'Hightail'},
 {'name': 'Nimbuzz'},
 {'name': 'IMVU'},
 {'name': 'Dropbox'},
 {'name': 'MobiTV'},
 {'name': 'Jimdo'},
 {'name': 'KIT digital'},
 {'name': 'BlueLithium'},
 {'name': 'MySpace'},
 {'name': 'ZoomInfo'},
 {'name': 'Zango'},
 {'name': 'Trulia'},
 {'name': 'TubeMogul'},
 {'name': 'TechTarget'},
 {'name': 'Sportsline'},
 {'name': 'OpenTable'},
 {'name': 'Espotting'},
 {'name': 'Adknowledge'},
 {'name': 'airG'},
 {

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.000

founded before 2000 &
acq amount gt 10.000.000

In [51]:
cond_1: {'founded_year': {'$lt': 2000}}
cond_2: {'acquisition': {'$gt':10000000}}
retrieve: {'_id':0, 'name':1}
list(col.find({
    '$and':
    [cond_1,cond_2]},{'name':1,'_id':0}).limit(10))


[{'name': 'AdventNet'},
 {'name': 'AddThis'},
 {'name': 'OpenX'},
 {'name': 'LifeLock'},
 {'name': 'Jajah'},
 {'name': 'Livestream'},
 {'name': 'Ustream'},
 {'name': 'iContact'},
 {'name': 'Yelp'},
 {'name': 'Dailymotion'}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [85]:
cond_1: {'acquisition.acquired_year':{'$gt':2010}}
retrieve = {'name':1, '_id':0, 'acquisitions':1}

list(col.find(cond_1,retrieve).limit(10))

[{'name': 'Wetpaint', 'acquisitions': []},
 {'name': 'Postini', 'acquisitions': []},
 {'name': 'Geni', 'acquisitions': []},
 {'name': 'Fox Interactive Media',
  'acquisitions': [{'price_amount': 20000000,
    'price_currency_code': 'USD',
    'term_code': None,
    'source_url': 'http://venturebeat.com/2007/05/30/fox-interactive-confirms-purchase-of-photobucket-and-flektor/',
    'source_description': 'Fox Interactive confirms purchase of Photobucket and Flektor',
    'acquired_year': 2007,
    'acquired_month': 5,
    'acquired_day': 30,
    'company': {'name': 'Flektor', 'permalink': 'flektor'}},
   {'price_amount': 700000000,
    'price_currency_code': 'USD',
    'term_code': 'cash',
    'source_url': None,
    'source_description': None,
    'acquired_year': 2005,
    'acquired_month': 9,
    'acquired_day': 1,
    'company': {'name': 'IGN Entertainment', 'permalink': 'ignentertainment'}},
   {'price_amount': None,
    'price_currency_code': 'USD',
    'term_code': None,
    'sourc

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [84]:
list(col.find({'name': {"$exists": True}},{'name':1, '_id':0,'founded_year':1}).sort('founded_year',-1).limit(10))

[{'name': 'Wamba', 'founded_year': 2013},
 {'name': 'Gimigo', 'founded_year': 2013},
 {'name': 'Clowdy', 'founded_year': 2013},
 {'name': 'Fluc', 'founded_year': 2013},
 {'name': 'Advaliant', 'founded_year': 2013},
 {'name': 'Pikk', 'founded_year': 2013},
 {'name': 'WhosCall', 'founded_year': 2013},
 {'name': 'SEOGroup', 'founded_year': 2013},
 {'name': 'iBazar', 'founded_year': 2013},
 {'name': 'Fixya', 'founded_year': 2013}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [60]:
cond_1 = {'founded_day':{'$lte': 7}}
retrieve = {'name':1,'_id':0}
list(col.find(cond_1,retrieve).sort('acquisition',-1).limit(10))

[{'name': 'Netscape'},
 {'name': 'PayPal'},
 {'name': 'Zappos'},
 {'name': 'Alibaba'},
 {'name': 'Postini'},
 {'name': 'Danger'},
 {'name': 'Clearwell Systems'},
 {'name': 'PrimeSense'},
 {'name': 'Amobee'},
 {'name': 'BlueLithium'}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [72]:
cond_1= {'category_code':'web'} 
cond_2= {'number_of_employees': {'$gt':4000}}
retrieve = {'name':1,'_id':0}
list(col.find(cond_1,retrieve).sort('number_of_employees',-1).limit(10))


[{'name': 'Experian'},
 {'name': 'eBay'},
 {'name': 'Yahoo!'},
 {'name': 'Rakuten'},
 {'name': 'Los Angeles Times Media Group'},
 {'name': 'Groupon'},
 {'name': 'Webkinz'},
 {'name': 'AOL'},
 {'name': 'Expedia'},
 {'name': 'Daum Communications'}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [89]:
cond_1 = {'acquisition.price_amount':{'$gt':10000000}}
cond_2 = {'acquisition.price_currency_code':'EUR'}
retrieve = {'name':1, '_id':0}
list(col.find({
    '$and':
    [cond_1,cond_2]},retrieve).limit(10))

[{'name': 'ZYB'},
 {'name': 'Apertio'},
 {'name': 'Greenfield Online'},
 {'name': 'Webedia'},
 {'name': 'Wayfinder'},
 {'name': 'Tuenti Technologies'},
 {'name': 'BioMed Central'}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [90]:
cond_1: {'acquisition.acquired_month':{'$gte':1}}
cond_2: {'acquisition.acquired_month':{'$lte':3}}
retrieve = {'name':1, '_id':0, 'acquisition':1}

list(col.find({
    '$and':
    [cond_1,cond_2]},retrieve).limit(10))

[{'name': 'ZYB',
  'acquisition': {'price_amount': 31500000,
   'price_currency_code': 'EUR',
   'term_code': 'cash',
   'source_url': 'http://www.techcrunch.com/2008/05/16/danish-mobile-social-network-zyb-acquired-by-vodafone-for-e315-million/',
   'source_description': 'Danish Network ZYB Acquired By Vodafone For â‚¬31.5 million',
   'acquired_year': 2008,
   'acquired_month': 5,
   'acquired_day': 16,
   'acquiring_company': {'name': 'Vodafone', 'permalink': 'vodafone'}}},
 {'name': 'Apertio',
  'acquisition': {'price_amount': 140000000,
   'price_currency_code': 'EUR',
   'term_code': None,
   'source_url': 'http://www.techcrunch.com/2008/01/03/nokia-siemens-acquires-apertio-for-e140m/',
   'source_description': 'Nokia Siemens Acquires Apertio For â‚¬140m',
   'acquired_year': 2008,
   'acquired_month': 1,
   'acquired_day': 3,
   'acquiring_company': {'name': 'Nokia Siemens Networks',
    'permalink': 'nokia-siemens-networks'}}},
 {'name': 'Greenfield Online',
  'acquisition': {'p

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code

**⚠️ Did you do this?**

```python
list(collection.find(query))[:5] #or a reasonably low number
```